<a href="https://colab.research.google.com/github/iremaricii/Ev-Fiyat-Tahmin-Sistemi/blob/main/FiyatTahmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle

In [ ]:
# Ames Housing
!kaggle datasets download -d prevek18/ames-housing-dataset
!unzip -o ames-housing-dataset.zip

# King County
!kaggle datasets download -d harlfoxem/housesalesprediction
!unzip -o housesalesprediction.zip

# SoCal görselli veri seti
!kaggle datasets download -d ted8080/house-prices-and-images-socal
!unzip -o house-prices-and-images-socal.zip

In [6]:
import pandas as pd
import numpy as np

# Ames Housing
ames = pd.read_csv("ameshousing.csv")
ames_df = ames[["SalePrice", "Gr Liv Area", "Garage Cars", "TotRms AbvGrd", "Year Built", "Overall Qual"]].dropna()

# King County
kc = pd.read_csv("kc_house_data.csv")
kc_df = kc[["price", "sqft_living", "bedrooms", "yr_built", "grade"]].copy()
kc_df.columns = ["SalePrice", "Gr Liv Area", "TotRms AbvGrd", "Year Built", "Overall Qual"]
kc_df["Garage Cars"] = np.random.randint(1, 4, size=kc_df.shape[0])
kc_df = kc_df[["SalePrice", "Gr Liv Area", "Garage Cars", "TotRms AbvGrd", "Year Built", "Overall Qual"]]

# Birleştirme
combined_df = pd.concat([ames_df, kc_df], ignore_index=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = combined_df.drop("SalePrice", axis=1)
y = combined_df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"R² Skoru: {r2_score(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")

In [ ]:
import matplotlib.pyplot as plt

combined_df["Year Built"] = combined_df["Year Built"].astype(int)
avg_by_year = combined_df.groupby("Year Built")["SalePrice"].mean()

plt.figure(figsize=(12,6))
plt.plot(avg_by_year.index, avg_by_year.values, marker="o")
plt.title("Yıllara Göre Ortalama Satış Fiyatı")
plt.xlabel("Yapım Yılı")
plt.ylabel("Fiyat (USD)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
socal_df = pd.read_csv("socal2.csv")
X_vis = socal_df[["bed", "bath", "sqft"]]
y_vis = socal_df["price"]

from sklearn.linear_model import LinearRegression
model_vis = LinearRegression()
model_vis.fit(X_vis, y_vis)

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def gorselli_tahmin(oda, banyo, alan):
    veri = np.array([[oda, banyo, alan]])
    tahmin = model_vis.predict(veri)[0]

    socal_df["distance"] = np.linalg.norm(X_vis.values - veri, axis=1)
    en_yakin = socal_df.sort_values("distance").iloc[0]

    # Doğru görsel yolu!
    image_path = f"socal2/socal_pics/{int(en_yakin['image_id'])}.jpg"

    bilgi = f"""📌 {en_yakin['street']} ({en_yakin['citi']})
🛏️ {int(en_yakin['bed'])} oda | 🛁 {int(en_yakin['bath'])} banyo
📐 {int(en_yakin['sqft'])} ft² | 💰 {int(en_yakin['price'])} USD"""

    return image_path, f"Tahmini Fiyat: {int(tahmin)} USD", bilgi

gr.Interface(
    fn=gorselli_tahmin,
    inputs=[
        gr.Slider(1, 6, value=3, step=1, label="Oda Sayısı"),
        gr.Slider(1, 4, value=2, step=1, label="Banyo Sayısı"),
        gr.Slider(500, 3000, value=1500, step=100, label="Yaşanabilir Alan (ft²)")
    ],
    outputs=[
        gr.Image(label="Benzer Ev Fotoğrafı"),
        gr.Textbox(label="Tahmini Fiyat"),
        gr.Textbox(label="Ev Bilgisi")
    ],
    title="Ev Fiyat Tahmin Sistemi",
    description="Girdiğiniz değerlere göre tahmini fiyatı görün ve en benzer evin fotoğrafını inceleyin."
).launch()
